In [163]:
import librosa
import joblib
import numpy as np
import matplotlib.pyplot as plt
import math

In [202]:
X = joblib.load("X_normalized.joblib")
y = joblib.load("y_normalized.joblib")

In [203]:
k = []
for item in X: 
    k.append(librosa.feature.spectral_bandwidth(item)[0])

In [176]:
q = []
for item in X:
    q.append(librosa.feature.spectral_contrast(item))

In [204]:
finallist = []
medlist = []
for item in k:
    if len(item)<110:
        diff = 110 - len(item)
        coppy = len(item) / diff
        for i in range (len(item)):
            if i % int(coppy) == 0:
                medlist.append(item[i])
                medlist.append(item[i])
            else:
                medlist.append(item[i])
            if len(medlist) == 110:
                finallist.append(medlist)
                medlist = []
                break
    elif len(item) == 110:
        finallist.append(item)
    else:
        diff = len(item) - 110
        remove = len(item) / diff
        ii = 0
        for i in range (len(item)):
            if ii % math.ceil(remove) == 0:
                avgitem = (item[ii] + item[ii+1])/2
                medlist.append(avgitem)
                ii = ii + 2
            else:
                medlist.append(item[ii])
                ii = ii + 1
            if len(medlist) == 110:
                finallist.append(medlist)
                medlist = []
                break

In [207]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(finallist,np.asanyarray(y), test_size=0.33, random_state=42)

In [208]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [209]:
model = Sequential()

model.add(Conv1D(128, 5,padding='same',
                 input_shape=(110,1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(8))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)

In [210]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=16, epochs=200, validation_data=(x_testcnn, y_test))